Author: **Soroosh Sanatkhani**\
Columbia University\
Created: February 2, 2023\
Last Modified: February 6, 2023

In [1]:
import os

# Mergin MRI with CT
### Input files are:
MRI_Original_corrected.nii.gz\
MRI_Skull_corrected.nii.gz\
CT_cropped.nii.gz

In [2]:
# Resampling MRI:
if(os.path.exists("./MRI.nii.gz")): os.remove("./MRI.nii.gz")
if(os.path.exists("./MRI_Skull.nii.gz")): os.remove("./MRI_Skull.nii.gz")

!flirt -in ./Inputs/MRI_Original_corrected.nii.gz -ref ./Inputs/MRI_Original_corrected.nii.gz -applyisoxfm 0.3 -nosearch -out MRI.nii.gz
!flirt -in ./Inputs/MRI_Skull_corrected.nii.gz -ref ./Inputs/MRI_Skull_corrected.nii.gz -applyisoxfm 0.3 -nosearch -out MRI_Skull.nii.gz

# Registering CT to MRI:
if(os.path.exists("./CT_registered.nii.gz")): os.remove("./CT_registered.nii.gz")
if(os.path.exists("./CT_registered.mat")): os.remove("./CT_registered.mat")
!flirt -in ./Inputs/CT_cropped.nii.gz -ref MRI_Skull.nii.gz -out CT_registered.nii.gz -omat CT_registered.mat -bins 256 -cost corratio -searchrx -90 90 -searchry -90 90 -searchrz -90 90 -dof 12 -interp trilinear
os.remove("./MRI_Skull.nii.gz")

# Resampling the Registered CT according to the MRI:
if(os.path.exists("./CT.nii.gz")): os.remove("./CT.nii.gz")
!flirt -in CT_registered.nii.gz -ref MRI.nii.gz -out CT.nii.gz -applyxfm -usesqform

# Combining the CT with MRI:
if(os.path.exists("./CT_Skull_mask.nii.gz")): os.remove("./CT_Skull_mask.nii.gz")
!fslmaths CT.nii.gz -bin CT_Skull_mask.nii.gz
!fslmaths CT_Skull_mask.nii.gz -sub 1 -abs CT_Skull_mask_inverted.nii.gz
!fslmaths CT.nii.gz -mul CT_Skull_mask.nii.gz CT_Skull.nii.gz
!fslmaths CT_Skull_mask_inverted.nii.gz -mul MRI.nii.gz -add CT_Skull.nii.gz MRI_CT.nii.gz
os.remove("./CT.nii.gz")
os.remove("./CT_Skull_mask_inverted.nii.gz")
os.remove("./CT_Skull.nii.gz")
os.remove("./MRI.nii.gz")
os.remove("./CT_registered.nii.gz")
os.remove("./CT_registered.mat")

# Creating parts:
### Input files are:
Outside.nii.gz\
T1w_mask_resample.nii.gz

In [5]:
!fslmaths ./Inputs/ITKSnap/Outside.nii.gz -kernel 3D -dilD Outside_dilated_1.nii.gz
!fslmaths Outside_dilated_1.nii.gz -kernel 3D -dilD Outside_dilated_2.nii.gz
os.remove("./Outside_dilated_1.nii.gz")
!fslmaths Outside_dilated_2.nii.gz -kernel 3D -dilD Outside_dilated_3.nii.gz
os.remove("./Outside_dilated_2.nii.gz")
!fslmaths Outside_dilated_3.nii.gz -kernel 3D -dilD Outside_dilated_4.nii.gz
os.remove("./Outside_dilated_3.nii.gz")
!fslmaths Outside_dilated_4.nii.gz -kernel 3D -dilD Outside_dilated_5.nii.gz
os.remove("./Outside_dilated_4.nii.gz")
if(os.path.exists("./Fitted/Skin_mask.nii.gz")): os.remove("./Fitted/Skin_mask.nii.gz")
!fslmaths Outside_dilated_5.nii.gz -sub ./Inputs/ITKSnap/Outside.nii.gz ./Fitted/Skin_mask.nii.gz
os.remove("./Outside_dilated_5.nii.gz")
!fslmaths ./Inputs/ITKSnap/Outside.nii.gz -sub 1 -abs  Inside.nii.gz
!fslreorient2std ./Inputs/T1w_mask_resample.nii.gz Brain_mask_corrected.nii.gz
!flirt -in Brain_mask_corrected -ref MRI_CT.nii.gz -out Brain_mask_registered.nii.gz -applyxfm -usesqform
os.remove("./Brain_mask_corrected.nii.gz")
!fslmaths Brain_mask_registered -bin Brain_mask
os.remove("./Brain_mask_registered.nii.gz")
!fslmaths Brain_mask -add CT_Skull_mask.nii.gz -bin Brain_plus_Skull
os.remove("./Brain_mask.nii.gz")
!3dmask_tool -input Brain_plus_Skull.nii.gz -prefix Brain_plus_Skull_filled.nii.gz -fill_holes -fill_dirs RL
os.remove("./Brain_plus_Skull.nii.gz")
if(os.path.exists("./Fitted/Muscle_mask.nii.gz")): os.remove("./Fitted/Muscle_mask.nii.gz")
if(os.path.exists("./Fitted/Brain_mask.nii.gz")): os.remove("./Fitted/Brain_mask.nii.gz")
!fslmaths Inside.nii.gz -sub Brain_plus_Skull_filled.nii.gz -sub ./Fitted/Skin_mask.nii.gz -bin ./Fitted/Muscle_mask.nii.gz
!fslmaths Brain_plus_Skull_filled.nii.gz -sub CT_Skull_mask.nii.gz -bin ./Fitted/Brain_mask.nii.gz
os.remove("./Brain_plus_Skull_filled.nii.gz")
if(os.path.exists("./Fitted/MRI_CT.nii.gz")): os.remove("./Fitted/MRI_CT.nii.gz")
!fslmaths MRI_CT.nii.gz -mul Inside.nii.gz ./Fitted/MRI_CT.nii.gz
os.remove("./MRI_CT.nii.gz")
if(os.path.exists("./Fitted/CT_Skull_mask.nii.gz")): os.remove("./Fitted/CT_Skull_mask.nii.gz")
!mv CT_Skull_mask.nii.gz ./Fitted/CT_Skull_mask.nii.gz
!mv Inside.nii.gz ./Fitted/Inside.nii.gz

/home/soroosh/fsl/bin/fslreorient2std: 1: bc: not found
++ no -frac option: defaulting to -union
++ processing 1 input dataset(s), NN=2...
++ padding all datasets by 0 (for dilations)
++ frac 0 over 1 volumes gives min count 0
++ voxel limits: 0 clipped, 3462227 survived, 94832567 were zero
++ filled 23541 holes (264202 voxels)
++ writing result Brain_plus_Skull_filled.nii.gz...
++ Output dataset ./Brain_plus_Skull_filled.nii.gz


# Cropping

In [6]:
!fslroi ./Fitted/CT_Skull_mask.nii.gz ./Cropped/CT_Skull_mask.nii.gz 1 279 138 317 290 190
!fslroi ./Fitted/MRI_CT.nii.gz ./Cropped/MRI_CT.nii.gz 1 279 138 317 290 190
!fslroi ./Fitted/Muscle_mask.nii.gz ./Cropped/Muscle_mask.nii.gz 1 279 138 317 290 190
!fslroi ./Fitted/Skin_mask.nii.gz ./Cropped/Skin_mask.nii.gz 1 279 138 317 290 190
!fslroi ./Fitted/Brain_mask.nii.gz ./Cropped/Brain_mask.nii.gz 1 279 138 317 290 190
!fslroi ./Fitted/Inside.nii.gz ./Cropped/Inside.nii.gz 1 279 138 317 290 190

# Cleaning
CLean the skin mask and inside using Slicer-->Segmentation-->remove islands-->export model and labels-->save .nii.gz and put them in ./Cleaned folder

In [7]:
!fslmaths ./Cropped/MRI_CT.nii.gz -mul ./Cleaned/Inside.nii.gz ./Cleaned/MRI_CT.nii.gz
!cp ./Cropped/Muscle_mask.nii.gz ./Cleaned/Muscle_mask.nii.gz
!cp ./Cropped/CT_Skull_mask.nii.gz ./Cleaned/CT_Skull_mask.nii.gz
!cp ./Cropped/Brain_mask.nii.gz ./Cleaned/Brain_mask.nii.gz

# Padding

In [4]:
if(os.path.exists("./Padded/MRI_CT.nii.gz")): os.remove("./Padded/MRI_CT.nii.gz")
if(os.path.exists("./Padded/Brain_mask.nii.gz")): os.remove("./Padded/Brain_mask.nii.gz")
if(os.path.exists("./Padded/CT_Skull_mask.nii.gz")): os.remove("./Padded/CT_Skull_mask.nii.gz")
if(os.path.exists("./Padded/Muscle_mask.nii.gz")): os.remove("./Padded/Muscle_mask.nii.gz")
if(os.path.exists("./Padded/Skin_mask.nii.gz")): os.remove("./Padded/Skin_mask.nii.gz")

!3dZeropad -prefix ./Padded/MRI_CT -L 100 -S 100 ./Cleaned/MRI_CT.nii.gz
!3dZeropad -prefix ./Padded/Brain_mask -L 100 -S 100 ./Cleaned/Brain_mask.nii.gz
!3dZeropad -prefix ./Padded/CT_Skull_mask -L 100 -S 100 ./Cleaned/CT_Skull_mask.nii.gz
!3dZeropad -prefix ./Padded/Muscle_mask -L 100 -S 100 ./Cleaned/Muscle_mask.nii.gz
!3dZeropad -prefix ./Padded/Skin_mask -L 100 -S 100 ./Cleaned/Skin_mask.nii.gz

!3dAFNItoNIFTI -prefix ./Padded/MRI_CT.nii.gz ./Padded/MRI_CT+orig
!3dAFNItoNIFTI -prefix ./Padded/Brain_mask.nii.gz ./Padded/Brain_mask+orig
!3dAFNItoNIFTI -prefix ./Padded/CT_Skull_mask.nii.gz ./Padded/CT_Skull_mask+orig
!3dAFNItoNIFTI -prefix ./Padded/Muscle_mask.nii.gz ./Padded/Muscle_mask+orig
!3dAFNItoNIFTI -prefix ./Padded/Skin_mask.nii.gz ./Padded/Skin_mask+orig

!rm ./Padded/*.BRIK
!rm ./Padded/*.BRIK.gz
!rm ./Padded/*.HEAD

++ 3dZeropad: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ output dataset: ./Padded/MRI_CT+orig.BRIK
++ 3dZeropad: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ output dataset: ./Padded/Brain_mask+orig.BRIK
++ 3dZeropad: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ output dataset: ./Padded/CT_Skull_mask+orig.BRIK
++ 3dZeropad: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ output dataset: ./Padded/Muscle_mask+orig.BRIK
++ 3dZeropad: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ output dataset: ./Padded/Skin_mask+orig.BRIK
++ 3dAFNItoNIFTI: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_22.3.07 (Dec  2 2022) [64-bit]
